In [13]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [23]:
path = '/Users/yeabinmoon/Documents/ibs_course/NLP/book/chapters/chapter_3/'
args = Namespace(
    raw_train_dataset_csv=path+"data/yelp/raw_train.csv",
    raw_test_dataset_csv=path+"data/yelp/raw_test.csv",
    proportion_subset_of_train=0.1,
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv=path+"data/yelp/reviews_with_splits_lite.csv",
    seed=1337
)

In [24]:
args.raw_train_dataset_csv

'/Users/yeabinmoon/Documents/ibs_course/NLP/book/chapters/chapter_3/data/yelp/raw_train.csv'

In [15]:
# Read raw data
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])

In [27]:
temp = pd.read_csv('/Users/yeabinmoon/Documents/ibs_course/NLP/book/chapters/chapter_3/data/yelp/raw_train.csv')
temp

,"<html lang=en><meta charset=utf-8><meta name=viewport content=""initial-scale=1",minimum-scale=1,"width=device-width""><title>Error 404 (Not Found)!!1</title><style nonce=""42V4qKx-JyAZUYlkMz-8KA"">*{margin:0;padding:0}html",code{font:15px/22px arial,"sans-serif}html{background:#fff;color:#222;padding:15px}body{color:#222;text-align:unset;margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px;}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}pre{white-space:pre-wrap;}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-moz-border-image:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) 0}}@media only screen and (-webkit-min-device-pixel-ratio:2){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat;-webkit-background-size:100% 100%}}#logo{display:inline-block;height:54px;width:150px}</style><main id=""af-error-container"" role=""main""><a href=//www.google.com><span id=logo aria-label=Google role=img></span></a><p><b>404.</b> <ins>That’s an error.</ins><p>The requested URL was not found on this server. <ins>That’s all we know.</ins></main>"


In [4]:
# making the subset equal across the review classes
by_rating = collections.defaultdict(list)
for _, row in train_reviews.iterrows():
    by_rating[row.rating].append(row.to_dict())
    
review_subset = []

for _, item_list in sorted(by_rating.items()):

    n_total = len(item_list)
    n_subset = int(args.proportion_subset_of_train * n_total)
    review_subset.extend(item_list[:n_subset])

review_subset = pd.DataFrame(review_subset)

In [5]:
review_subset.head()

""


In [8]:
train_reviews

,,,rating,review
"<html lang=en><meta charset=utf-8><meta name=viewport content=""initial-scale=1",minimum-scale=1,"width=device-width""><title>Error 404 (Not Found)!!1</title><style nonce=""42V4qKx-JyAZUYlkMz-8KA"">*{margin:0;padding:0}html",code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;pad...


In [6]:
train_reviews.rating.value_counts()

2    280000
1    280000
Name: rating, dtype: int64

In [7]:
review_subset.rating.value_counts()

2    28000
1    28000
Name: rating, dtype: int64

In [8]:
# Unique classes
set(review_subset.rating)

{1, 2}

In [9]:
# Splitting the subset by rating to create our new train, val, and test splits
by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())
    
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):

    np.random.shuffle(item_list)
    
    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)
    
    # Give data point a split attribute
    for item in item_list[:n_train]:
        item['split'] = 'train'
    
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
        
    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'

    # Add to final list
    final_list.extend(item_list)

In [10]:
# Create split data
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):

    np.random.shuffle(item_list)
    
    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)
    
    # Give data point a split attribute
    for item in item_list[:n_train]:
        item['split'] = 'train'
    
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
        
    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'

    # Add to final list
    final_list.extend(item_list)

In [11]:
# Write split data to file
final_reviews = pd.DataFrame(final_list)

In [12]:
final_reviews.split.value_counts()

train    39200
test      8400
val       8400
Name: split, dtype: int64

In [13]:
# Preprocess the reviews
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text
    
final_reviews.review = final_reviews.review.apply(preprocess_text)

In [14]:
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)

In [15]:
final_reviews.head()

,rating,review,split
0,negative,all i can say is that a i had no other option ...,train
1,negative,i went here once when my long time stylist mov...,train
2,negative,i don t know why i stopped here for lunch this...,train
3,negative,did i order the wrong thing ? or maybe it was ...,train
4,negative,i went here for restaurant week . the restaura...,train


In [16]:
final_reviews.to_csv(args.output_munged_csv, index=False)

In [28]:
import json
import pandas as pd
data_file = open("yelp_academic_dataset_checkin.json")
data = []
for line in data_file:
&nbsp;data.append(json.loads(line))
checkin_df = pd.DataFrame(data)
data_file.close()

IndentationError: expected an indented block (672583651.py, line 6)